In [109]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

import pandas as pd
import numpy as np

In [95]:
df_train = pd.read_csv('train.csv')

df_test = pd.read_csv('test.csv')

print(df_train.columns)
print(df_test.columns)

Index(['label', 'pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5',
       'pixel6', 'pixel7', 'pixel8',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=785)
Index(['pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5', 'pixel6',
       'pixel7', 'pixel8', 'pixel9',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=784)


In [96]:
print(df_train.shape)
print(df_test.shape)

(42000, 785)
(28000, 784)


In [97]:
X = df_train
X = X.drop(columns=['label'])

print(X.shape)

(42000, 784)


In [98]:
X = X.values
X_test = df_test.values

X = X.dot(1/255)
X_test = X_test.dot(1/255)

X = X.reshape(X.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

print(X_test.shape)
print(X.shape)

(28000, 28, 28, 1)
(42000, 28, 28, 1)


In [99]:
Y = df_train['label'].values

print(Y.shape)

(42000,)


In [100]:
from keras.utils import np_utils

Y = np_utils.to_categorical(Y)

print(Y.shape)

(42000, 10)


In [28]:
print(Y[-1])

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]


In [29]:
from sklearn.model_selection import train_test_split

random_seed = 7

split_train_x, split_test_x, split_train_y, split_test_y, = train_test_split(X, Y, test_size = 0.10, random_state=random_seed)
split_train_x, split_val_x, split_train_y, split_val_y, = train_test_split(split_train_x, split_train_y, test_size = 0.10, random_state=random_seed)

In [30]:
reduce_learning_rate = ReduceLROnPlateau(monitor='val_acc',
                              factor=0.3,
                              patience=3,
                              min_lr=0.0001)

In [31]:
imgGenerator = ImageDataGenerator( rotation_range=10,
                              zoom_range = 0.1,
                              width_shift_range=0.1,
                              height_shift_range=0.1)
imgGenerator.fit(split_train_x)

In [135]:
def CNN(label_class_size):
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))


    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(label_class_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
    return model

In [136]:
model = CNN(Y.shape[1])

In [137]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_35 (Conv2D)           (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 28, 28, 32)        25632     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 7, 7, 64)          0         
__________

In [139]:

history = model.fit_generator(imgGenerator.flow(split_train_x,split_train_y, batch_size=64),
                    epochs = 1, validation_data = (split_val_x,split_val_y),
                    verbose = 2, steps_per_epoch=640, callbacks=[reduce_learning_rate])

Epoch 1/1


 - 158s - loss: 0.4002 - acc: 0.8744 - val_loss: 0.0881 - val_acc: 0.9735


In [140]:
from sklearn.metrics import accuracy_score, recall_score, precision_score

predictions_train = model.predict(split_train_x, verbose=0)
predictions_val = model.predict(split_val_x, verbose=0)
predictions_test = model.predict(split_test_x, verbose=0)


In [141]:
print("Resultados CNN: ")
print('-> Acuracia de treino: ', accuracy_score(predictions_train.round(), split_train_y))
print('-> Recall de treino: ', recall_score(predictions_train.round(), split_train_y, average='weighted'))
print('-> Precisão de treino: ', precision_score(predictions_train.round(), split_train_y, average='weighted'))

print('-> Acuracia de validacao: ', accuracy_score(predictions_val.round(), split_val_y))
print('-> Recall de validacao: ', recall_score(predictions_val.round(), split_val_y, average='weighted'))
print('-> Precisão de validacao: ', precision_score(predictions_val.round(), split_val_y, average='weighted'))

print('-> Acuracia de teste: ', accuracy_score(predictions_test.round(), split_test_y))
print('-> Recall de teste: ', recall_score(predictions_test.round(), split_test_y, average='weighted'))
print('-> Precisão de teste: ', precision_score(predictions_test.round(), split_test_y, average='weighted'))

Resultados CNN: 


-> Acuracia de treino:  0.973780129336
-> Recall de treino:  0.981628540951
-> Precisão de treino:  0.974073131089
-> Acuracia de validacao:  0.97037037037
-> Recall de validacao:  0.977351452172
-> Precisão de validacao:  0.970593484675
-> Acuracia de teste:  0.975714285714
-> Recall de teste:  0.981792045999
-> Precisão de teste:  0.975975062395


In [157]:
model = CNN(Y.shape[1])
history = model.fit_generator(imgGenerator.flow(split_train_x,split_train_y, batch_size=64),
                    epochs = 30, validation_data = (split_test_x,split_test_y),
                    verbose = 2, steps_per_epoch=640, callbacks=[reduce_learning_rate])

Epoch 1/30


 - 164s - loss: 0.4007 - acc: 0.8705 - val_loss: 0.2682 - val_acc: 0.9233


Epoch 2/30


 - 160s - loss: 0.1123 - acc: 0.9668 - val_loss: 0.0438 - val_acc: 0.9876


Epoch 3/30


...

 - 162s - loss: 0.0179 - acc: 0.9945 - val_loss: 0.0259 - val_acc: 0.9921


Epoch 29/30


 - 160s - loss: 0.0188 - acc: 0.9944 - val_loss: 0.0241 - val_acc: 0.9929


Epoch 30/30


 - 160s - loss: 0.0192 - acc: 0.9941 - val_loss: 0.0251 - val_acc: 0.9929


In [158]:
predictions_train = model.predict(X, verbose=0)
print('-> Acuracia de total: ', accuracy_score(predictions_train.round(), Y))
print('-> Recall de total: ', recall_score(predictions_train.round(), Y, average='weighted'))
print('-> Precisão de total: ', precision_score(predictions_train.round(), Y, average='weighted'))

-> Acuracia de total:  0.996904761905
-> Recall de total:  0.997237174296
-> Precisão de total:  0.99690672012


In [159]:
predictions_sub = model.predict_classes(X_test)

In [160]:
print(predictions_sub[-1])

2


In [161]:
submit = pd.DataFrame(data=predictions_sub, columns=('Label',), index=np.arange(1, predictions_sub.shape[0]+1))

In [161]:
submit.to_csv('submitions.csv')

In [ ]:
# Plot the loss and accuracy curves for training and validation 
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['acc'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_acc'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

NameError: name 'inline' is not defined